In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Store csv created in part one into a DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,aureliana,10.89,121.98,79.20,73,90,4.34,PH,1585692931
1,1,bethel,41.37,-73.41,48.20,61,90,6.93,US,1585692931
2,2,faget,45.85,22.18,27.00,79,94,5.82,RO,1585692932
3,3,mataura,-46.19,168.86,53.01,98,23,1.90,NZ,1585692932
4,4,hervey bay,-25.30,152.85,73.99,94,20,6.93,AU,1585692932


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=300,
                                 point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) 
                                    & (city_data_df["Cloudiness"] == 0)]

narrowed_city_df = narrowed_city_df.dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
87,87,road town,18.42,-64.62,78.01,75,0,5.99,VG,1585692959
90,90,inhambane,-23.86,35.38,74.14,70,0,9.10,MZ,1585692960
117,117,cockburn town,21.46,-71.14,77.14,68,0,9.75,TC,1585692968
172,172,katwa,23.64,88.13,76.24,23,0,2.82,IN,1585692983
194,194,maheshwar,22.18,75.58,73.40,40,0,5.82,IN,1585692989
243,243,porbandar,21.64,69.61,73.99,88,0,9.98,IN,1585693000
287,287,capao bonito,-24.01,-48.35,71.44,64,0,5.26,BR,1585693012
332,332,nyaunglebin,17.95,96.73,70.12,84,0,2.73,MM,1585693024
337,337,chabahar,25.29,60.64,72.64,70,0,9.98,IR,1585693025
352,352,colgong,25.27,87.22,76.55,22,0,2.39,IN,1585693029


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
87,road town,VG,18.42,-64.62,
90,inhambane,MZ,-23.86,35.38,
117,cockburn town,TC,21.46,-71.14,
172,katwa,IN,23.64,88.13,
194,maheshwar,IN,22.18,75.58,
243,porbandar,IN,21.64,69.61,
287,capao bonito,BR,-24.01,-48.35,
332,nyaunglebin,MM,17.95,96.73,
337,chabahar,IR,25.29,60.64,
352,colgong,IN,25.27,87.22,


In [14]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    params["location"] = f'{row["Lat"]},{row["Lng"]}'

    hotel_name = requests.get(url, params=params).json()

    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print(f"Missing result for city: {row['City']}")

hotel_df

Missing result for city: obo


,City,Country,Lat,Lng,Hotel Name
87,road town,VG,18.42,-64.62,Sea View Hotel
90,inhambane,MZ,-23.86,35.38,Hotel Casa Do Capitão
117,cockburn town,TC,21.46,-71.14,Osprey Beach Hotel
172,katwa,IN,23.64,88.13,Utsav Inn
194,maheshwar,IN,22.18,75.58,Ahilya Fort
243,porbandar,IN,21.64,69.61,Hotel Sheetal
287,capao bonito,BR,-24.01,-48.35,HOTEL PASSARIM
332,nyaunglebin,MM,17.95,96.73,Sann Htay Guest House
337,chabahar,IR,25.29,60.64,Lipar Hotel
352,colgong,IN,25.27,87.22,SPOT ON 61925 Hotel Prerna


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
marker = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker)

fig

Figure(layout=FigureLayout(height='420px'))